In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from fastai.text.all import *
import torch
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Пример работы токенайзера

In [6]:
ids = tokenizer.encode('Это пример текста')
ids

[4441, 2433, 13124]

In [7]:
tokenizer.decode(ids)

'Это пример текста'

Генерация ответа

In [ ]:
t = torch.LongTensor(ids)[None]
preds = model.generate(t)

In [10]:
preds.shape,preds[0]

(torch.Size([1, 20]),
 tensor([ 4441,  2433, 13124,    16,  1029,   417,   322,  2447,  7869,    16,
           515,  1029,   417,  2447,  7869,    16,   753,   387, 10668,    16]))

In [11]:
tokenizer.decode(preds[0].numpy())

'Это пример текста, который я не могу привести, но который я могу привести, чтобы вы поняли,'

Открываем датасет с анекдотами на русском языке

In [19]:

with open("extract_dialogues_from_anekdots.txt", 'r',encoding='utf-8') as file:
    file_content = file.read()


Разделяем текст на отдельные диалоги

In [20]:
data = file_content.split("\n\n\n")

In [31]:
all_texts = data

In [66]:
splits = [range_of(data), list(range(len(data), len(all_texts)))]

Делаем батч сайз равным 4, а максимальное число токенов равным 256 (в гпт2 использовалось 1024)

In [67]:
bs,sl = 4,256

Токенизируем текст

In [40]:
def tokenize(text):
    toks = tokenizer.tokenize(text)
    return tensor(tokenizer.convert_tokens_to_ids(toks))

tokenized = [tokenize(t) for t in progress_bar(all_texts)]

В методе encodes мы учитываем случай, когда мы получаем что-то, что еще не было токенизировано, на случай, если мы будем создавать набор данных с новыми текстами, используя это преобразование.

In [41]:
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        return x if isinstance(x, Tensor) else tokenize(x)
        
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

Применяем с помощью встроеной функции TfmdLists методы из TransformersTokenizer 

In [42]:
tls = TfmdLists(tokenized, TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)
dls = tls.dataloaders(bs=bs, seq_len=sl)

In [43]:
dls.show_batch(max_n=2)

,text,text_
0,"\n- Скажи, дорогой, как это ты, такой умный и красивый, женился именно на мне?\n- А, тебя это тоже удивляет...\n- Мойше, ты слышал, что внук Иоськи спит с мальчиками???\n- Что ты говоришь, но хотя бы с еврейскими??\n- Фамилия?\n- Чья?\n- Имя?\n- Чье?\n- Чья Чье? Мексиканец, что ли?\n- Скажите, вы выпиваете?\n- Да, но это потом, сначала о деле.\n- Я здесь вчера ужинал?\n- Тут.\n- А 10 тысяч долларов я у вас пропил7 - У нас.\n- Ф-фу... Слава Богу, а я думал, что потерял!\n- Доктор, мой муж импотент на все 200 процентов!\n- Как так?! Ну на сто процентов я понимаю...\n- Так оно и было, но вчера он запнулся и прикусил язык.\n- Я выпил бы что-нибудь безалкогольное.\n- Лимонад, сок, минеральная вода?\n- Мне все равно, я в этой области дилетант...\n- Скажите, кто это? В.И. -...\n- Мы вам подска","- Скажи, дорогой, как это ты, такой умный и красивый, женился именно на мне?\n- А, тебя это тоже удивляет...\n- Мойше, ты слышал, что внук Иоськи спит с мальчиками???\n- Что ты говоришь, но хотя бы с еврейскими??\n- Фамилия?\n- Чья?\n- Имя?\n- Чье?\n- Чья Чье? Мексиканец, что ли?\n- Скажите, вы выпиваете?\n- Да, но это потом, сначала о деле.\n- Я здесь вчера ужинал?\n- Тут.\n- А 10 тысяч долларов я у вас пропил7 - У нас.\n- Ф-фу... Слава Богу, а я думал, что потерял!\n- Доктор, мой муж импотент на все 200 процентов!\n- Как так?! Ну на сто процентов я понимаю...\n- Так оно и было, но вчера он запнулся и прикусил язык.\n- Я выпил бы что-нибудь безалкогольное.\n- Лимонад, сок, минеральная вода?\n- Мне все равно, я в этой области дилетант...\n- Скажите, кто это? В.И. -...\n- Мы вам подскажем"
1,"обвиняли в ограблении магазина.\n- Да? Как интересно! И какой срок ты ему дал, любимый?\n- Пришлось освободить. У него оказалось железное алиби: в это время он грабил ба\n- Василий Иваныч, что такое ""сексуальная прелюдия""?\n- Да это, Петька, то, что у нас с Анкой только на людях.\n- Доктор, а ходить я буду?\n- Конечно будете,...под себя.\n- Ну, как там наша Мэри поживает?\n- Она умерла.\n- Умерла?!!\n- Да, умерла от гонореи.\n- Да ну, люди не умирают от гонореи!\n- Умирают, если ты заразила ею БОЛЬШОГО ДЖО....\n- Вахтанг, что такоэ Тыбет?\n- Э, Гоги, Тыбет это то же что и минэт, толко нэ минэ а тыбе.\n- Что было между тобой и соседом?\n- Ничего, даже пижамы.\n- Папа, что такое ""критика сверху"" и ""критика снизу""?\n- Сейчас обьясню... Папа берет табуретку, становится на нее, и - ТЬФУ!","яли в ограблении магазина.\n- Да? Как интересно! И какой срок ты ему дал, любимый?\n- Пришлось освободить. У него оказалось железное алиби: в это время он грабил ба\n- Василий Иваныч, что такое ""сексуальная прелюдия""?\n- Да это, Петька, то, что у нас с Анкой только на людях.\n- Доктор, а ходить я буду?\n- Конечно будете,...под себя.\n- Ну, как там наша Мэри поживает?\n- Она умерла.\n- Умерла?!!\n- Да, умерла от гонореи.\n- Да ну, люди не умирают от гонореи!\n- Умирают, если ты заразила ею БОЛЬШОГО ДЖО....\n- Вахтанг, что такоэ Тыбет?\n- Э, Гоги, Тыбет это то же что и минэт, толко нэ минэ а тыбе.\n- Что было между тобой и соседом?\n- Ничего, даже пижамы.\n- Папа, что такое ""критика сверху"" и ""критика снизу""?\n- Сейчас обьясню... Папа берет табуретку, становится на нее, и - ТЬФУ!\n"


# Дообучение на новых данных

Здесь нам нужно написать событие after_pred и заменить self.learn.pred (содержащий предсказания, которые будут переданы функции потерь) только его первым элементом.

In [44]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [45]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

Получаем результат validation loss = 3.2, perplexity = 25.7

In [ ]:
learn.validate()

In [48]:
learn.fit_one_cycle(1, 1e-4)

epoch,train_loss,valid_loss,perplexity,time
0,3.151185,3.160435,23.580856,53:03


In [68]:
prompt = "Чтобы сдать экзамен по искуственному интеллекту нужно"

In [69]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None]
inp.shape

torch.Size([1, 8])

In [ ]:
preds = learn.model.generate(inp, max_length=20, num_beams=5, temperature=1.5)

In [71]:
tokenizer.decode(preds[0].cpu().numpy())

'Чтобы сдать экзамен по искуственному интеллекту нужно, чтобы его не было?\n- Да, конечно,'